In [1]:
# Import libraries
import pandas as pd
import pandas_ta as ta
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns

In [2]:
# Vars
timeframe = "H1"
symbols = ["GBP_JPY"]
systems = ["Strat"]
starting_balance = 100
risk_per_trade = 0.02 # Risking 2% per trade
market_open_time = "09:00"
opening_range_hours = 5
latest_entry_time = "22:00"
trade_direction = "long"
timezone = "Europe/London"

In [ ]:
# Load CSV File
def get_price_data(symbol):
    df = pd.read_csv(f"../data/{symbol}_{timeframe}.csv", parse_dates=['Datetime'], index_col='Datetime')
    df.index = pd.to_datetime(df.index, utc=True).tz_convert(timezone)  # Keeps local timezone without converting to UTC
    return df